## Classification

#### Classification is the process of classifying the objects from the input image using Classifier Network.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Tanh(),

            nn.Linear(120, 84),
            nn.Tanh(),

            nn.Linear(84, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)

        return x

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081)) 
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [5]:
num_epochs = 5

In [6]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 99:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

    # Test loop after each epoch
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            predicted = output.argmax(dim=1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy after epoch {epoch + 1}: {accuracy:.2f}%\n')

print("Training complete!")

Epoch [1/5], Batch [100/938], Loss: 0.8910
Epoch [1/5], Batch [200/938], Loss: 0.2732
Epoch [1/5], Batch [300/938], Loss: 0.1968
Epoch [1/5], Batch [400/938], Loss: 0.1325
Epoch [1/5], Batch [500/938], Loss: 0.1347
Epoch [1/5], Batch [600/938], Loss: 0.1028
Epoch [1/5], Batch [700/938], Loss: 0.1002
Epoch [1/5], Batch [800/938], Loss: 0.0941
Epoch [1/5], Batch [900/938], Loss: 0.0904
Accuracy after epoch 1: 97.85%

Epoch [2/5], Batch [100/938], Loss: 0.0654
Epoch [2/5], Batch [200/938], Loss: 0.0686
Epoch [2/5], Batch [300/938], Loss: 0.0645
Epoch [2/5], Batch [400/938], Loss: 0.0613
Epoch [2/5], Batch [500/938], Loss: 0.0707
Epoch [2/5], Batch [600/938], Loss: 0.0678
Epoch [2/5], Batch [700/938], Loss: 0.0640
Epoch [2/5], Batch [800/938], Loss: 0.0596
Epoch [2/5], Batch [900/938], Loss: 0.0563
Accuracy after epoch 2: 98.38%

Epoch [3/5], Batch [100/938], Loss: 0.0457
Epoch [3/5], Batch [200/938], Loss: 0.0420
Epoch [3/5], Batch [300/938], Loss: 0.0471
Epoch [3/5], Batch [400/938], Los